# Loading data from colorhunt

In [3]:
from urllib import request,parse
import json
import time
import random
from tqdm import trange

def load():
    data = []
    for i in trange(0,64,desc='Loading data'):
        body = parse.urlencode({'step':i,'sort':'new','tags':'','timeframe':30}).encode()
        Req = request.Request('https://colorhunt.co/php/feed.php',method='POST',headers={'content-type': 'application/x-www-form-urlencoded','User-Agent' :'Mozilla/4.0 (compatible; MSIE 5.5; Windows NT)'})
        readed = request.urlopen(Req,data=body).read()
        loads = json.loads(readed)
        data += loads
        time.sleep(random.randint(5,10)/100) # slow down to prevent http 429
    return data

with open('dataset/color.json','w') as f:
    f.write(json.dumps(load()))

Loading data: 100%|██████████| 64/64 [00:37<00:00,  1.70it/s]


# Process data



In [113]:
import json
import time
from tqdm import trange
import random

def color_code_split(code):
    return [f'#{code[i:i+6]}'.upper() for i in range(0,len(code),6)]

raw_data = []

with open('dataset/color.json','r') as f:
    raw_data = json.loads(f.read())

data = []
for i in trange(len(raw_data),desc="Transform color codes"):
    hex = color_code_split(raw_data[i]['code']) # transform color_code in data to hex
    for l in range(len(hex)):
        row = [str(i*4+l),hex[l],'|'.join(hex)] # create palette id
        row.append(raw_data[i]['likes'])
        data.append(row)
    # time.sleep(0.0001)

with open('dataset/color_processed.csv','w') as f:
    f.write("id,base,palette,likes\n" + '\n'.join([','.join(i) for i in data]))

Transform color codes: 100%|██████████| 2546/2546 [00:00<00:00, 5534.92it/s]


# Training

In [10]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

batch_size = 64

def hex_2_rbg(hex):
    hex = hex.replace('##','#')
    r = (int('0x'+ hex[1:3],16))
    g = (int('0x'+ hex[3:5],16))
    b = (int('0x'+ hex[5:],16))
    return [r,g,b]


df = pd.read_csv('dataset/color_processed.csv')
df['base'] = df['base'].map(lambda hex: np.asarray([hex_2_rbg(hex)]))
df['palette'] = df['palette'].map(lambda hexs: np.asarray([[hex_2_rbg(i) for i in hexs.strip().split("|")]]))

x = []
y = []
for i in df['id']:
    x.append(df['base'][i])
    y.append(df['palette'][i])

train_x, test_x, train_y, test_y = train_test_split( np.array(x) , np.array(y) , test_size=0.1 )
dataset = tf.data.Dataset.from_tensor_slices( ( train_x , train_y ) )
dataset = dataset.batch( batch_size )
